# California kindergarten vaccination analysis

By Sandra Poindexter and [Ben Welsh](mailto:ben.welsh@latimes.com)

This notebook documents that analysis conducted for the August 13, 2017, _Los Angeles Times_ story ["Despite California's strict new law, hundreds of schools still don't have enough vaccinated kids."](http://www.latimes.com/health/la-me-kindergarten-vaccination-20170813-htmlstory.html)

Even with a new law that has boosted kindergarten vaccination rates to record highs, hundreds of schools across California still have so many children lacking full immunization that they pose an increased risk of disease outbreaks, the Times analysis of state data. This was caused in part by a dramatic increase in doctors signing notes that allow students to skip their shots.

## How we did it

The collection, transformation and analysis of the records that underpin the story are presented below.

### Import Python tools

In [42]:
import re
import warnings
import numpy as np
import pandas as pd

In [43]:
warnings.simplefilter("ignore")

In [2]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

### Download source data

The California Department of Public Health has published school-level data in Microsoft Excel format. The data is provided in two tabs. They are imported and combined below.

In [3]:
raw_sheet1_df = pd.read_excel(
    "https://archive.cdph.ca.gov/programs/immunize/Documents/2016-17_CA_Kindergarten_Data.xlsx",
    sheetname="Enrollment 20 or More",
    skiprows=3,
    names=[
        'school_code',             
        'county',              
        'public_or_private',              
        'district',            
        'city',
        'name',           
        'enrollment',            
        'number_up_to_date',             
        'percent_up_to_date',             
        'number_conditional_entrant',            
        'percent_conditional_entrant',            
        'number_permanent_medical_exemption',             
        'percent_permanent_medical_exemption',             
        'number_personal_belief_exemption',             
        'percent_personal_belief_exemption',             
        'number_others',    
        'percent_others',    
        'number_overdue',         
        'percent_overdue',         
        'number_dtp',             
        'percent_dtp',             
        'number_polio',           
        'percent_polio',           
        'number_mmr',             
        'percent_mmr',             
        'number_hepb',            
        'percent_hepb',            
        'number_varicella',             
        'percent_varicella',             
        'reported'                
    ],
    dtype={
        'school_code': str  # The state's school identifier has leading zeroes so this is important.
    }
)

Replace the state's null values with the proper pandas' objects

In [4]:
raw_sheet1_df.replace("--*", np.NaN, inplace=True)

Label the source sheet of these records

In [5]:
raw_sheet1_df['sheet_name'] = 'Enrollment 20 or More'

Kill out any null rows, which should also removes the footnotes at the bottom of the file.

In [6]:
raw_sheet1_df = raw_sheet1_df[~raw_sheet1_df.county.isnull()]

Import the second tab, which lists schools with under 20 students. Their results were withheld by the state.

In [8]:
raw_sheet2_df = pd.read_excel(
    "https://archive.cdph.ca.gov/programs/immunize/Documents/2016-17_CA_Kindergarten_Data.xlsx",
    sheetname="Enrollment 19 or Less",
    skiprows=4,
    names=[
        'school_code',             
        'county',              
        'public_or_private',              
        'district',            
        'city',
        'name',           
        'enrollment',            
        'reported'                
    ],
    dtype={'school_code':str}
)

Label this sheet

In [9]:
raw_sheet2_df['sheet_name'] = 'Enrollment 19 or Less'

Combine the two sheets into one dataframe

In [10]:
raw_df = pd.concat([raw_sheet1_df, raw_sheet2_df], axis=0)

### Prepare the data file for analysis

Trim the dataset down to only those schools with reported data and 20 or more students. The schools left out do not have any data to analyze.

In [11]:
analysis_df = raw_df[
    (raw_df.reported == 'Y') &
    (raw_df.enrollment >= 20)
]

Next we convert the data values published by the state into integers we can analyze.

As a "precaution for student de-identification" the state rounded off values when percentages surpassed thresholds at or above 95% of school enrollment. In response, our analysis will use a conservative 90% immunization rate as the threshold to group and classify schools.

In [12]:
def clean_ints(val):
    """
    Accepts a value string and converts it to an integer.
    
    Values that have been obscured as values like ≥95 and ≤5 are returned as 95 and 5.
    """
    # If the value is already a float, just return it.
    if isinstance(val, float):
        return val
    # If it's null, return a NaN object
    if pd.isnull(val):
        return np.NaN
    # Otherwise, try to parse out the integer from the state's string value
    number = re.search(r'\d+', val)
    # If we find an integer, convert it and return the value
    if number:
        return int(number.group(0))
    # Otherwise return a NaN null value
    return np.NaN

Convert the data columns we will analyze into integers

In [15]:
analysis_df.loc[:, 'percent_up_to_date_int'] = analysis_df.percent_up_to_date.apply(clean_ints)

In [17]:
analysis_df.loc[:, 'percent_permanent_medical_exemption_int'] = analysis_df.percent_permanent_medical_exemption.apply(clean_ints)

### Analyze the data

The dataset is now prepared for analysis. Next we will present the findings from the story alongside the code that supports each claim.

#### Finding: The analysis included about 6,500 schools with 20 or more kindergartners.

In [18]:
len(analysis_df)

6469

#### Finding: At nearly 750 schools, 90% or fewer kindergardeners had been fully vaccinated last year

In [19]:
under_90 = analysis_df[analysis_df.percent_up_to_date_int <= 90]

In [20]:
len(under_90)

739

#### Finding: In the school year that began last fall, the law's first year, the number of kindergartners in California with medical exemptions tripled, the analysis found.

This finding is drawn from a summary report published by the California Department of Public Health. It cannot be reproduced with this dataset because the state has withheld some values.

![](./input/pme_totals.png)

In [21]:
2850 / 931.0

3.061224489795918

#### Finding: At 58 schools, 10% or more kindergartners had medical exemptions last fall. The rate topped 20% at seven schools.

First with the 10 percent cutoff

In [22]:
over_10_pme = analysis_df[analysis_df.percent_permanent_medical_exemption_int >= 10]

In [23]:
len(over_10_pme)

58

Again with 20% as the cutoff

In [24]:
over_20_pme = analysis_df[analysis_df.percent_permanent_medical_exemption_int >= 20]

In [25]:
len(over_20_pme)

7

#### Finding: At Sebastopol Independent Charter in Sonoma County, 11 of 45 kindergartners had a medical exemption from vaccination last school year. An additional 17 had personal belief exemptions that rolled over from the previous year. All together, only about a third of the class was fully vaccinated, among the lowest rates in the state.

In [26]:
analysis_df[analysis_df.name == 'SEBASTOPOL INDEPENDENT CHARTER'][[
    'name',
    'county',
    'enrollment',
    'number_up_to_date',
    'number_permanent_medical_exemption',
    'number_personal_belief_exemption',
    'percent_up_to_date'
]]

,name,county,enrollment,number_up_to_date,number_permanent_medical_exemption,number_personal_belief_exemption,percent_up_to_date
6375,SEBASTOPOL INDEPENDENT CHARTER,SONOMA,45.0,12,11,17,27


#### Finding: Statewide, private and charter schools account for the majority of schools where 90% or fewer kindergartners had received all their shots.

This finding requires merging in the state's public school roster, which indicates which public schools are charter operated. That file was previously downloaded from [this page](http://www.cde.ca.gov/ds/si/ds/pubschls.asp).

In [27]:
public_school_roster = pd.read_csv("./input/public_schools.csv", dtype={'school_code': str})

Merge that with our list of schools with a vaccination rate lower than 90%.

In [28]:
under_90_with_type = pd.merge(
    under_90,
    public_school_roster[['school_code', 'is_charter']],
    on="school_code",
    how="left"
)

Classify schools as public, private or charter.

In [29]:
def get_school_type(row):
    """
    Classifies schools as public, private or charter.
    """
    # If the row is null, return a NaN
    if pd.isnull(row.public_or_private):
        return np.NaN
    # If the vaccination data says the school is private, we will classify it as private
    if row.public_or_private == 'PRIVATE':
        return 'private'
    # If the school is public ...
    if row.public_or_private == 'PUBLIC':
        # ... if the public school roster says it's a charter, we classify it that way
        if row.is_charter == 'Y':
            return 'charter'
        # ... otherwise call it a public school
        else:
            return 'public'

In [31]:
under_90_with_type['school_type'] = under_90_with_type.apply(get_school_type, axis=1)

Count under 90 schools by type

In [32]:
under_90_by_type = under_90_with_type.school_type.value_counts().reset_index()

In [33]:
under_90_by_type['percent'] = under_90_by_type.school_type / under_90_by_type.school_type.sum()

In [34]:
under_90_by_type

,index,school_type,percent
0,public,362,0.485255
1,private,209,0.280161
2,charter,175,0.234584


#### Finding: A third of the schools with low vaccination rates where in Los Angeles County, followed by San Diego and Orange. 

Regroup the same dataset by county

In [35]:
under_90_by_county = under_90.groupby("county").agg(dict(school_code="count")).reset_index()

In [36]:
under_90_by_county['percent'] = under_90_by_county.school_code / under_90_by_county.school_code.sum()

In [37]:
under_90_by_county.sort_values("percent", ascending=False).head()

,county,school_code,percent
13,LOS ANGELES,241,0.326116
28,SAN DIEGO,61,0.082544
22,ORANGE,56,0.075778
26,SACRAMENTO,42,0.056834
27,SAN BERNARDINO,25,0.033829


#### Finding: Seven of 46 kindergartners at Innovations Academy had medical exemptions last year. State data show there were no exemptions in the previous year.

In [38]:
analysis_df[analysis_df.name == 'INNOVATIONS ACADEMY'][[
    'name',
    'county',
    'enrollment',
    'number_up_to_date',
    'number_permanent_medical_exemption',
    'number_personal_belief_exemption',
    'percent_up_to_date'
]]

,name,county,enrollment,number_up_to_date,number_permanent_medical_exemption,number_personal_belief_exemption,percent_up_to_date
5034,INNOVATIONS ACADEMY,SAN DIEGO,46.0,35,7,0,76


### Export data for online search

Finally we output data to power a searchable database at [spreadsheets.latimes.com/vaccination-rates-1617/](http://spreadsheets.latimes.com/vaccination-rates-1617/).

Trim the columns downj to the ones we want to keep.

In [39]:
search_df = raw_df[[
    'name',
    'county',
    'enrollment',
    'percent_permanent_medical_exemption',
    'percent_up_to_date',
]]

Substitute in a human readable null value.

In [44]:
search_df.fillna("N/A", inplace=True)

In [45]:
search_df.percent_permanent_medical_exemption = search_df.percent_permanent_medical_exemption.str.replace(".", "N/A").str.strip()
search_df.percent_up_to_date = search_df.percent_up_to_date.str.replace(".", "N/A").str.strip()

Convert the obscured values into HTML entities.

In [46]:
search_df.percent_permanent_medical_exemption = search_df.percent_permanent_medical_exemption.str.replace("%", "")
search_df.percent_up_to_date = search_df.percent_up_to_date.str.replace("%", "")

In [47]:
search_df.percent_permanent_medical_exemption = search_df.percent_permanent_medical_exemption.str.replace("≤", "&lt;")
search_df.percent_up_to_date = search_df.percent_up_to_date.str.replace("≥", "&gt;")

In [48]:
search_df.percent_permanent_medical_exemption = search_df.percent_permanent_medical_exemption.str.replace("≤", "&lt;")
search_df.percent_up_to_date = search_df.percent_up_to_date.str.replace("≥", "&gt;")

Convert the column headers into more readable language.

In [49]:
search_df.columns = [
    "Name",
    "County",
    "Enrollment",
    "Medical exemptions (%)",
    "Up to date (%)"
]

Write the data out to a comma-delimited file

In [50]:
search_df.sort_values(["County", "Name"]).to_csv("./output/search.csv", index=False, encoding="utf-8")